## Config

In [1]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

# Filepaths
dataPath = os.path.join('data', 'artistsKnown.csv')

## Log In

In [2]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Collect new artists

In [4]:
# Collect names of new artists from Music Festival Wizard
artistsNew = fetchMusicFestivalWizard()
artistsNew = artistsNew.head()
artistsNew

,artistName
1,21 Savage
2,3 Doors Down
3,311
4,A Day To Remember
5,A Tribe Called Quest


In [3]:
# Collect artists from an existing favorited playlist
playlistToFetch = getPlaylistID(credentials, 'anti pop')
artistsNew = fetchTracksFromPlaylist(credentials, playlistToFetch)
artistsNew

,artistName,artistID,songName,release_date,songURI,songLocation,albumLength,songDuration,songExplicit,songPopularity
0,sagacious,0wzoSAAL4Yq5ph8q28HXEU,ur self,2021-08-17,spotify:track:6tfoSWteKyW43Cf0Usdywm,1,1,176094,False,34
1,Tommy Newport,7FvQR6QFfXmW18dyMqnHvv,Jonny's Habit,2023-02-17,spotify:track:09YcDy6I05RGXAq6gPDVr0,1,10,197014,False,35
2,Mercury,6aIZIwesnYKR6ZN1fIerPZ,heaven,2023-06-01,spotify:track:5gEoep9RwLEy8hFJMne9ad,1,1,120133,True,30
3,Portraits Of Tracy,34QizBjxVeMcA5HlZpKxxE,Found!,2023-05-26,spotify:track:1FgY491AT8WYmMgZgWLfkz,16,18,305511,False,48
4,Billy Lemos,7ebBg3BuRFa2satTcY8whC,No Place Here (feat. annonXL),2023-02-24,spotify:track:5gAPeHJRjeTirQxYQT2qeE,1,7,116156,False,30
...,...,...,...,...,...,...,...,...,...,...
120,Sophia Black,3wdXgSpjbbU3tX9KasdxMh,goddammit.,2023-04-28,spotify:track:005w7f7Ect0chkLpfGmYvx,1,3,178493,True,43
121,King Isis,7jz1wM1NdiskFWrlosZhVw,taste of u,2023-03-01,spotify:track:4zqZ7hR4Pl3UT0p19En1ib,1,3,179358,False,42
122,Nana Lourdes,3LEjiFgpLbT1F0UgOE9qbg,Made Your Mama Cry,2021-06-30,spotify:track:3GROKC4eYt3KVWN3AyMCwL,1,1,158451,False,45
123,Ray Laurél,7gW3GsnBSoT6q9YQUstlzA,DONTLEAVEMYSIDE,2023-04-12,spotify:track:5dx52a9oh4Vm7XbkqCn7Dq,1,1,126965,False,43


## Process new artists

In [5]:
# Filter out known artists who we have already listened to
if os.path.exists(dataPath):
    artistsKnown = pd.read_csv(dataPath, index_col=0)
else:
    artistsKnown = pd.DataFrame(columns=['artistName', 'artistID', 'artistPopularity', 'artistFollowers', 'artistGenre'])
artistsNew = artistsNew.loc[~artistsNew.artistName.isin(artistsKnown['artistName'])]


,artistName


In [9]:
# Search Spotify for new artist info
# artistsNew = artistsNew.apply(fetchArtistInfo, axis=1)
# artistsNew

spotify, userID, user_token = credentials
possibleArtists, = spotify.search('21 Savage', types=['artist'], limit=1)
possibleArtists.items[0]

FullArtist with fields:
  external_urls = {'spotify'}
  followers = Followers(href, total)
  genres = [4 x str]
  href = 'https://api.spotify.com/v1/artists/1URnnhqYAYcrqrcwql10ft'
  id = '1URnnhqYAYcrqrcwql10ft'
  images = [3 x Image(height, url, width)]
  name = '21 Savage'
  popularity = 90
  type = 'artist'
  uri = 'spotify:artist:1URnnhqYAYcrqrcwql10ft'

In [ ]:
# Search spotify for artists based on name
possibleArtists, = spotify.search(artistsNew['artistName'], types=['artist'], limit=1)
# Select the first search result
artistsNew['artistID'] = possibleArtists.items[0].id
artists['artistPopularity'] = possibleArtists.items[0].popularity
artists['artistFollowers'] = possibleArtists.items[0].followers.total
artists['artistGenre'] = ", ".join(list(possibleArtists.items[0].genres))
result = artists

In [ ]:
# Ignore artists who did not turn up in Spotify
artistsNew = artistsNew.dropna(subset='artistID')
# Drop artists whose name was not an exact match with a known artist but who have the same artistID
artistsNew = artistsNew.loc[~artistsNew.artistID.isin(artistsKnown['artistID'])]

## Get top songs

In [83]:
# Get their top three songs in last 5 years
songURIs = fetchTopSongs(artistsNew['artistID'], maxAge=5)
# How many songs are in this round?
print(f"There are {len(songURIs)} songs about to be added")

There are 6 about to be added


## Put songs in playlist

In [114]:
# Warning: This will make a new playlist
playlistID = getPlaylistID('New Artists to Explore')
# Add songs to the playlist
addToPlaylist(songURIs, playlistID)

## Update the list of known artists

In [126]:
# Append artistsNew to artistsKnown.csv 
allArtists = pd.concat([artistsKnown, artistsNew], axis=0, ignore_index=True)
allArtists.to_csv(dataPath)

## Create playlist to drop good songs into

In [118]:
# Cull exploreArtists into ArtistHasPotential playlist
winnersPlaylist = getPlaylistID('New Artists I Liked')
print(f'Now drop songs you like into this playlist: {winnersPlaylist}')

Now drop songs you like into this playlist: 5bZPe8Gkc9jUADiDXXlm4V
